In [ ]:
from parser import parse_config, data_converter, sample_set
from visualize_letters import print_letter, generate_noisy_samples, add_noise_to_letter
import matplotlib.pyplot as plt
import numpy as np
from optimized_autoencoder import OptimizedAutoencoder

config_json = parse_config("resources/config.json")
activation_function = (lambda x: 1/(1+np.exp(-2*x)))
activation_derivative = (lambda x, *args: 2*(1/(1+np.exp(-2*x)))*(1-(1/(1+np.exp(-2*x)))) )
denoising_config = config_json["denoising"]
# noise_prob = denoising_config["noise_prob"]
sample_size = denoising_config["sample_size"]
noise_probabilities = denoising_config["multiple_noise_prob"]
font_letters=list()
training_set=list()
expected_output = list()
for i in config_json["font_numbers"]:
 
    print("New letter====")
    labels, alphabet = data_converter(f"resources/fonts_{i}.txt")
    indexes = np.random.randint(0, len(alphabet), 2)
    for idx in indexes:
        font_letters.append(np.array(alphabet[idx]))
        for p in noise_probabilities:
            noisy_letters = generate_noisy_samples(np.array(alphabet[idx]), sample_size, p)
            flattened_noisy_letters = np.array(list(map(lambda char: np.array(char).flatten(), noisy_letters)))
            training_set.extend(flattened_noisy_letters)
            for j in range(sample_size):
                expected_output.append(np.array(alphabet[idx]).flatten())
    
autoencoder = OptimizedAutoencoder(optimizer="powell", 
input_dim=len(training_set[0]), latent_dim=5, hidden_layers=[25, 17], 
activation=activation_function, activation_derivative=activation_derivative)

# for i in range(sample_size*len(noise_probabilities)):
#     expected_output.append(np.array(alphabet[idx]).flatten())
# print(expected_output)
print(len(training_set))
print(len(expected_output))
autoencoder.train(training_set,expected_output , max_iterations=20)

for let in font_letters:
    print_letter(let)
    for p in noise_probabilities:
        noisy_letter = add_noise_to_letter(let, p)
        print_letter(noisy_letter)
        output, err = autoencoder.reconstruct(noisy_letter.flatten(), let.flatten())
        # print("output: "+str(output))
        print_letter(output.reshape(noisy_letters[0].shape))
        print(f"Error: {err}")





